In [6]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals

import numpy as np
from keras.models import load_model
import sys
sys.path.insert(0, '../../../')
from prepare_data import *

from keras import optimizers
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

In [7]:
############################ LOCATION ########################
letter_location_model = load_model('../cnn_dictionary/v3_10/20_test_epoch.h5', custom_objects={'actual_accuracy': actual_accuracy})

# num_examples = len(data.x_train)  # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20

conv_input_shape=(10, 5168)
othr_input = (140, )
conv_input = Input(shape=conv_input_shape, name='conv_input')

# syllabled letters
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)

syllable_location_model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
syllable_location_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
syllable_location_model.load_weights('../syllables/v2_4/20_epoch.h5')


conv_input_shape=(10, 252)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')

# syllabled letters
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)


syllabled_letters_location_model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
syllabled_letters_location_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
syllabled_letters_location_model.load_weights('../syllabled_letters/v2_5_3/20_test_epoch.h5')

In [96]:
############################ TYPE ########################
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20

# letters
conv_input_shape=(23, 36)
othr_input = (150, )
conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
x = concatenate([x_conv, othr_input])
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)


letter_type_model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
letter_type_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
letter_type_model.load_weights('../../accent_classification/letters/v2_1/20_test_epoch.h5')


In [36]:
# generates text with every word in new line
with open('../../../../reldi-tagger/test_original') as f:
    original_text = f.readlines()
original_text = ''.join(original_text)
# print(original_text)
text_with_whitespaces = original_text.replace(',', ' ,').replace('.', ' .').replace('\n', ' ').replace("\"", " \" ").replace(":", " :").replace("ć", "č").replace('–', '-')
# print('-------------------------------------------------')
text_with_whitespaces = '\n'.join(text_with_whitespaces.split())
text_with_whitespaces += '\n\n'
# print(text_with_whitespaces)
with open('../../../../reldi-tagger/test_with_whitespaces', "w") as text_file:
    text_file.write(text_with_whitespaces)
    
# generates text with PoS tags
import subprocess

myinput = open('../../../../reldi-tagger/test_with_whitespaces', 'r')
myoutput = open('../../../../reldi-tagger/test_result', 'w')
# print(myinput.readlines())
python3_command = "../../../../reldi-tagger/tagger.py sl"  # launch your python2 script using bash

process = subprocess.run(python3_command.split(), stdin=myinput, stdout=myoutput)

# generates interesting words
pointless_words = ['.',',','\"',':', '-']
with open('../../../../reldi-tagger/test_result', "r") as text_file:
    tagged_input_words = []
    for x in text_file.readlines()[:-1]:
        splited_line = x[:-1].split('\t')
        if splited_line[0] not in pointless_words and not any(char.isdigit() for char in splited_line[0]):
            tagged_input_words.append([splited_line[0].lower(), '', splited_line[1], splited_line[0].lower()])
#     tagged_input_words = [x[:-1].split('\t') for x in text_file.readlines() if x[:-1].split('\t')[0] not in pointless_words][:-1]

In [16]:
data = Data('l', shuffle_all_inputs=False)

In [14]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [97]:
# X, X_other_features, fake_y = generate_X_and_y(dictionary, max_word, max_num_vowels, tagged_input_words, vowels, accetuated_vowels, feature_dictionary, 'who cares', shuffle=False)
%run ../../../prepare_data.py
pos = 45
print(tagged_input_words[pos])

data = Data('l', shuffle_all_inputs=False)
x, x_other_features, fake_y = data._generate_x_and_y(dictionary, max_word, max_num_vowels, tagged_input_words, vowels, accented_vowels, feature_dictionary, 'who cares')
generator = data._letter_generator(x, x_other_features, fake_y, batch_size, accented_vowels)
letter_location_predictions = letter_location_model.predict_generator(generator, len(x)/(batch_size))

data = Data('s', shuffle_all_inputs=False)
x, x_other_features, fake_y = data._generate_x_and_y(syllable_dictionary, max_word, max_num_vowels, tagged_input_words, vowels, accented_vowels, feature_dictionary, 'who cares')
eye = np.eye(len(syllable_dictionary), dtype=int)
generator = data._syllable_generator(x, x_other_features, fake_y, batch_size, eye, accented_vowels)
syllable_location_predictions = syllable_location_model.predict_generator(generator, len(x)/(batch_size))


data = Data('sl', shuffle_all_inputs=False)
x, x_other_features, fake_y = data._generate_x_and_y(syllable_dictionary, max_word, max_num_vowels, tagged_input_words, vowels, accented_vowels, feature_dictionary, 'who cares')
max_syllable = data._get_max_syllable(syllable_dictionary)
syllable_letters_translator = data._create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)
generator = data._syllable_generator(x, x_other_features, fake_y, batch_size, syllable_letters_translator, accented_vowels)
syllabled_letters_location_predictions = syllabled_letters_location_model.predict_generator(generator, len(x)/(batch_size))


predictions = np.mean( np.array([ letter_location_predictions, syllable_location_predictions, syllabled_letters_location_predictions ]), axis=0 )




# data = Data('l', shuffle_all_inputs=False, accent_classification=True)
# x, x_other_features, fake_y = data._generate_x_and_y(dictionary, max_word, max_num_vowels, tagged_input_words, vowels, accented_vowels, feature_dictionary, 'who cares')
# generator = data._letter_generator(x, x_other_features, fake_y, batch_size, accented_vowels)
# letter_type_predictions = letter_type_model.predict_generator(generator, len(x)/(batch_size))


# letter_type_model





#predictions = model.predict([X, X_other_features])
print(data.decode_y(predictions[pos]))
print(predictions[pos])

words_and_accetuation_loc = [[tagged_input_words[i][0], data.decode_y(predictions[i])] for i in range(len(tagged_input_words))]

original_text_list = list(original_text)
original_text_lowercase = original_text.lower()
end_pos = 0
for word in words_and_accetuation_loc:
    posit = original_text_lowercase.find(word[0], end_pos)
    if posit != -1:
        start_pos = posit
        end_pos = start_pos + len(word[0])
    
    original_text_list[start_pos:end_pos] = list(accetuate_word(''.join(original_text_list[start_pos:end_pos]), word[1]))

    
    #print(start_pos)
#    if 4005 < start_pos < 4028:
#        print(word[0])
#        print(original_text_list[start_pos:end_pos])
#        print(start_pos)
    
#    if word[0] == "zalivu":
#        print("TEST")
#        print(start_pos)
#        print(end_pos)
#        print(original_text_list[start_pos:end_pos])
#        print(''.join(original_text_list))
    
print(''.join(original_text_list))
#print(original_text_list[4006:4027])

['zaliva', '', 'Ncmsg', 'zaliva']
[1]
[  6.30854402e-06   9.99771297e-01   5.57102567e-06   9.06875703e-12
   1.12715172e-15   3.22355225e-20   2.57569017e-26   1.88833358e-21
   1.21982240e-26   0.00000000e+00]
Urádni podátki polícije kažejó na précej napéte razmére v Piránskem zalívu, jé dánes poročála oddája 24UR. Do dánes se jé zgodílo žé 36 incídentov, v katérih só hrváška plovíla 52 krát zaplúla v vodé, ki veljájo kot obmóčje Slovenskéga nadzóra, toréj prekó črté, ki oznáčuje trí četŕtine Piranskéga zalíva.

Slovénska polícija sícer zagótavlja, da né povzróča incídentov in oprávlja nalóge enáko kot préd arbitrážno razsódbo. Hrváška strán pa na vprašánje, zákaj vztrajnó plujé do sredíne zalíva, né odgóvarja.

Ribíči pojasnújejo, da na obmóčju, kjér se dogajájo incídenti, lovíjo prédvsem listé, hobotníce in kalámare. Ríbič Zlatkó Novógradec jé za 24UR déjal, da si vsí skupáj želíjo čimpréjšnjo impleméntacijo razsódbe: "Zdáj smó kot sardíne v konzérvi in s tistím koščkóm morjá bomó 

In [3]:
# evaluate the model
scores = model.evaluate([X_test, X_other_features_test], y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

51840/52058 [============================>.] - ETA: 0s
actual_accuracy: 86.97%


In [4]:
def test_accuracy(predictions, y):
#     dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
    num_of_pred = len(predictions)
    num_of_correct_pred = 0
    for i in range(predictions.shape[0]):
        if decode_position(predictions[i]) == decode_position(y[i]):
            num_of_correct_pred += 1

    return (num_of_correct_pred/float(num_of_pred)) * 100

predictions = model.predict([X_test, X_other_features_test])
print(test_accuracy(predictions, y_test))

86.97414422375043


In [5]:
incorrects = np.nonzero(predictions != y_test)

In [44]:
test = np.equal(predictions,y_test)
print(predictions[0])
print(y_test[0])
results = [[accetuate_word(decode_input(X_test[i], dictionary), decode_position(predictions[i])), predictions[i]] for i in range(len(predictions)) if decode_position(predictions[i]) != decode_position(y_test[i])]
print(len(results))
print(results[1])

[  0.00000000e+00   1.00000000e+00   6.09418273e-23   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
6781
['spregovórjena', array([  0.00000000e+00,   4.67960648e-09,   3.46557694e-09,
         9.99999046e-01,   7.49402659e-07,   4.68860552e-23,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00], dtype=float32)]


In [9]:

dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
feature_dictionary = create_feature_dictionary()

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


In [39]:
if 'A' not in vowels:
    vowels.extend(['A', 'E', 'I', 'O', 'U'])
print(vowels)
def accetuate_word(word, locations):
#     word = list(word)
    word_list = list(word)
    for loc in locations:
        vowel_num = 0
        #if loc == 0:
        #    return word
        for i in range(len(word_list)):
            if data._is_vowel(word_list, i, vowels):
                if word_list[i] == 'a' and vowel_num == loc:
                    word_list[i] = 'á'
                elif word_list[i] == 'e' and vowel_num == loc:
                    word_list[i] = 'é'
                elif word_list[i] == 'i' and vowel_num == loc:
                    word_list[i] = 'í'
                elif word_list[i] == 'o' and vowel_num == loc:
                    word_list[i] = 'ó'
                elif word_list[i] == 'u' and vowel_num == loc:
                    word_list[i] = 'ú'
                elif word_list[i] == 'r' and vowel_num == loc:
                    word_list[i] = 'ŕ'
                elif word_list[i] == 'A' and vowel_num == loc:
                    word_list[i] = 'Á'
                elif word_list[i] == 'E' and vowel_num == loc:
                    word_list[i] = 'É'
                elif word_list[i] == 'I' and vowel_num == loc:
                    word_list[i] = 'Í'
                elif word_list[i] == 'O' and vowel_num == loc:
                    word_list[i] = 'Ó'
                elif word_list[i] == 'U' and vowel_num == loc:
                    word_list[i] = 'Ú'
                elif word_list[i] == 'R' and vowel_num == loc:
                    word_list[i] = 'Ŕ'
                vowel_num += 1
#     print(word_list)
    return ''.join(word_list)
    
accetuate_word('policije', [3])

['à', 'á', 'ä', 'é', 'ë', 'ì', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü', 'a', 'e', 'i', 'o', 'u', 'A', 'E', 'I', 'O', 'U']


'policijé'

In [10]:
%run ../../../prepare_data.py
X, X_other_features, fake_y = generate_X_and_y(dictionary, max_word, max_num_vowels, tagged_input_words, vowels, accetuated_vowels, feature_dictionary, 'who cares', shuffle=False)

pos = 18
print(tagged_input_words[pos])
predictions = model.predict([X, X_other_features])
print(decode_position(predictions[pos]))
print(predictions[pos])



words_and_accetuation_loc = [[tagged_input_words[i][0], decode_position(predictions[i])] for i in range(len(tagged_input_words))]

CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
['je', '', 'Va-r3s-n', 'je']
[0, 1]
[  7.99477637e-01   6.77873671e-01   9.83008295e-02   6.05765490e-05
   6.33550807e-12   2.99607361e-09   1.19315509e-08   1.73794911e-06
   8.25467694e-09   8.27788467e-13   4.58576916e-13]


In [13]:
original_text_list = list(original_text)
original_text_lowercase = original_text.lower()
end_pos = 0
for word in words_and_accetuation_loc:
    start_pos = original_text_lowercase.find(word[0], end_pos)
    end_pos = start_pos + len(word[0])
    original_text_list[start_pos:end_pos] = list(accetuate_word(''.join(original_text_list[start_pos:end_pos]), word[1]))

print(''.join(original_text_list))

Urádni podátki policíje kážejo ná precéj napéte razmére v Piránskem zalívu, je danés poróčala oddája 24UR. Dó danés se je zgodílo žé 36 incidéntov, v katérih so hrváška plovíla 52 krat zaplúla v vóde, ki veljájo kot obmóčje Slovénskega nadzóra, tórej préko čŕte, ki označúje trí četrtíne Piránskega zalíva.

Slovénska policíja sícer zagotávlja, da ne povzróča incidéntov in oprávlja nalóge enáko kot préd arbitrážno razsódbo. Hrváška strán pa ná vprašánje, zakáj vztrájno plúje dó sredíne zalíva, ne odgovárja.

Ríbiči pojasnújejo, da ná obmóčju, kjér se dogájajo incidénti, lovíjo predvsém líste, hobótnice in kalámare. Ríbič Zlátko Novográdec je za 24UR dejál, da si vsí skúpaj želíjo čimpréjšnjo implementácijo razsódbe: "Zdáj smo kot sardíne v konzérvi in s tístim kóščkom mórja bómo mogóče le málo zadíhali in zagotóvo se bó káka ríba véč znášla v mréži."

Vprašánje pa je túdi, kakó bó v priméru incidéntov ríbiče zaščítila náša policíja. Téh informácij túdi ríbiči sámi – nímajo.

Erjávec pozí

In [177]:
words_and_accetuation_loc

[['uradni', [2]],
 ['podatki', [2]],
 ['policije', [3]],
 ['kažejo', [1]],
 ['na', [1]],
 ['precej', [2]],
 ['napete', [2]],
 ['razmere', [2]],
 ['v', [1]],
 ['piranskem', [2]],
 ['zalivu', [2]],
 ['je', [0, 1]],
 ['danes', [2]],
 ['poročala', [2]],
 ['oddaja', [2]],
 ['do', [1]],
 ['danes', [2]],
 ['se', [0]],
 ['je', [0, 1]],
 ['zgodilo', [2]],
 ['že', [1]],
 ['incidentov', [3]],
 ['v', [1]],
 ['katerih', [2]],
 ['so', [0, 1]],
 ['hrvaška', [2]],
 ['plovila', [2]],
 ['krat', []],
 ['zaplula', [2]],
 ['v', [1]],
 ['vode', [1]],
 ['ki', [0]],
 ['veljajo', [2]],
 ['kot', [0]],
 ['območje', [2]],
 ['slovenskega', [2]],
 ['nadzora', [2]],
 ['torej', [1]],
 ['preko', [1]],
 ['črte', [1]],
 ['ki', [0]],
 ['označuje', [3]],
 ['tri', [1]],
 ['četrtine', [3]],
 ['piranskega', [2]],
 ['zaliva', [2]],
 ['slovenska', [2]],
 ['policija', [3]],
 ['sicer', [1]],
 ['zagotavlja', [3]],
 ['da', [0]],
 ['ne', [0, 1]],
 ['povzroča', [2]],
 ['incidentov', [3]],
 ['in', [0]],
 ['opravlja', [2]],
 ['naloge'